# ENVIRONMENT

In [13]:
#First tradingEnv class
#todo add transaction cost
# add MACD indicator
# add RSI indicator
# add SMA indicator
# add EMA indicator
# meerdere crypto's toevoegen ( momenteel alleen BTC)
import numpy as np
import pandas as pd
class TradingEnv():
    def __init__(self, initial_investment=20000, Terminal_state= 10000):
        #investment
        self.initial_investment = initial_investment #initial investment 
        self.Terminal_state = Terminal_state  #terminal state, when the agent reach this state or below, the episode is over
        self.current_investment = initial_investment #current investment in a certain state
        self.non_invested = initial_investment #non invested money in a certain state
        self.invested = 0
        self.worth_history = [] #worth history in a certain episode
        # self.invested = [{'amount': 0, 'price_investment': 0, 'timestamp' : UNIX timestamp of investment}] #invested money 
        self.invested_amount = 0
        
        #rewards
        self.current_reward = 0 #current reward in a certain state
        self.total_reward = 0   #total reward in a certain episode
        self.reward_history = [] #reward history in a certain episode
        self.iterations = 0

        #states
        self.current_price = [0,0,0,0] #current price in a certain state [open,high,low,close]
        self.prev_price = np.zeros(20) #previous 20 prices to keep in memory to the state values
        self.done = False #if the episode is over
        self.current_unix_time = 0 #current unix time in a certain state
        self.macd = 0 #macd indicator
        
        
        #actions
        self.action = 0 #chosen action in a certain state
        self.action_history = [] #action history in a certain episode
        self.action_space = np.array([0,1,2]) #action space, 0 = hold, 1 = buy, 2 = sell first in the array 

        #Current state
        # self.current_state = self.get_current_state() #current state
        self.csv_to_dataframe() #convert csv's to one dataframe
        self.csv_index = 0 #index of the csv dataframe
        self.demo = False

    def get_current_state(self):
        #return current state price, and invested/non invested values as state
        return [self.current_price[0],self.current_price[1],self.current_price[2],self.current_price[3], self.invested, self.non_invested, self.invested_amount, self.macd,self.sma30,self.sma60, self.lastBuy, self.lastSold, self.lastBuyPrice, self.lastSoldPrice]
    # def calculate_macd(self):
    #     #calculate macd indicator
    #     print("macd")
    #     # print(self.df.iloc[self.csv_index - 12 : self.csv_index, 3])
    #     # print(self.df.iloc[self.csv_index - 12 : self.csv_index, 3].ewm(span=12, adjust=False).mean())
    #     macd = self.df.iloc[self.csv_index - 500 : self.csv_index, 3].ewm(span=500, adjust=False).mean()
    #     macd = macd.tolist()
    #     print('res',macd)
    #     # print('res',type(macd.tolist()))
    #     print("average macd", sum(macd) / len(macd))

        return macd
    def reset(self, demo=False):
        # self.lastBuy, self.lastSold, self.lastBuyPrice, self.lastSoldPrice
        self.lastBuy = -9999
        self.lastSold = -9999
        self.lastBuyPrice = 0
        self.lastSoldPrice = 0
        
        self.counter = 0
        #reset the environment
        self.worth_history = []
        self.current_investment = self.initial_investment
        self.current_reward = 0
        self.total_reward = 0
        self.reward_history = []
        self.current_price = [0,0,0,0] #current price in a certain state [open,high,low,close]
        self.invested_amount = 0
        self.non_invested = self.initial_investment #non invested money in a certain state
        self.invested = 0
        self.worth_history = [] #worth history in a certain episode

        self.prev_price = np.zeros(20)
        self.action = 0
        self.action_history = []
        self.done = False
        self.current_unix_time = 0 #current unix time in a certain state
        self.csv_index = 0 #index of the csv dataframe
        self.demo = demo

        #begin somewhere random in csv data 
        
        if demo == False:
            #last 6months as test set
            self.csv_index = 2100000
        else:
            self.csv_index = len(self.df) - 262975
        #get row csv_index from self.df
        self.current_price = self.df.iloc[self.csv_index, 1:5].values
        self.current_unix_time = self.df.iloc[self.csv_index, 0]
        
        self.macd =  self.df.iloc[self.csv_index, -1]
        self.sma60 =  self.df.iloc[self.csv_index, -5]
        self.sma30 =  self.df.iloc[self.csv_index, -4]






        return self.get_current_state(), self.done

    def step(self, action, interval = 1):
        self.counter +=1
        self.lastBuy -= 1
        self.lastSold -= 1
        self.iterations += 1
        self.action = action
        self.action_history.append(self.action)
        
        #calculate current reward
        self.current_reward = self.get_reward()
        self.done = self.is_done()
        #if the episode is over, give a big negative reward
        if self.done == True:
            self.current_reward - 10000000

        self.total_reward += self.current_reward
        self.reward_history.append(self.current_reward)

        #get next line of data in csv
        if self.csv_index < len(self.df) - 100:
            self.csv_index += interval
            self.current_price = self.df.iloc[self.csv_index, 1:5].values

            self.current_unix_time = self.df.iloc[self.csv_index, 0]
            self.macd =  self.df.iloc[self.csv_index, -1]
            self.sma60 =  self.df.iloc[self.csv_index, -5]
            self.sma30 =  self.df.iloc[self.csv_index, -4]

        else:
            self.done = True
        # if self.csv_index > len(self.df) - 262975 and self.demo == False:
        #     self.done = True
        #     print("end of DF")
        
        return self.get_current_state(), self.current_reward, self.done
        
    def get_reward(self):
        #reward function
        if self.action == 0:
            #hold do nothing
            return -5
        elif self.action == 1:
            if self.invested > ((self.calc_current_worth()/4)*3):
                return -25
                # self.done = True
            elif self.lastBuy > -60 or self.lastSold > -40:
                return -25
                # self.done = True
            else:
                #buy = invest 1/10 of non_invested money
                to_invest = self.non_invested / 5
                self.non_invested -= to_invest
                amount = to_invest / self.current_price[0]
                self.invested += to_invest
                self.invested_amount += amount
                self.lastBuy = 0
                self.lastBuyPrice = self.current_price[0]
                reward = 1750
        elif self.action == 2:
            #sell first bought in array of invested
            if self.lastBuy > -10 and self.lastSold > -20:
                reward = -25
            elif self.invested > 10:
                amount = self.invested_amount
                price_investment = self.invested
                self.non_invested += amount * self.current_price[0]
                self.invested_amount = 0
                self.invested = 0
                self.lastSold = 0
                self.lastSoldPrice = self.current_price[0]
                if ((amount * self.current_price[0]) - price_investment) > 0:
                    reward = (((amount * self.current_price[0]) - price_investment))* 500
                else:
                    reward = (((amount * self.current_price[0]) - price_investment)) * -100
            else:
                #hold, do nothing, nothing to sell
                return -25
        
        self.worth_history.append(self.calc_current_worth())
        if reward > 0: 
            return (reward) * ((self.calc_current_worth()/self.initial_investment)**2)
        else:
            return (reward) * ((self.calc_current_worth()/self.initial_investment)**2)



    def is_done(self):
        #terminal state
        if self.calc_current_worth() <= self.Terminal_state:
            print("terminal state reached")
            return True
        else:
            return False

    def calc_current_worth(self):
        #calculate current worth
        return self.invested_amount * self.current_price[0] + self.non_invested

        
    def csv_to_dataframe(self):
        #convert csv's to one dataframe
        # df2017 = pd.read_csv('../data/BTC-2017min.csv')
        # df2018 = pd.read_csv('../data/BTC-2018min.csv')
        # df2019 = pd.read_csv('../data/BTC-2019min.csv')
        # df2020 = pd.read_csv('../data/BTC-2020min.csv')
        # df2021 = pd.read_csv('../data/BTC-2021min.csv')

        #concat all dataframes
        # df = pd.concat([df2017, df2018, df2019, df2020, df2021], ignore_index=True)

        df = pd.read_csv('./data/ETHUSD_1.csv', header=None)

        #add new column named macd, that is the mean of the last 12 open values
        #OHLCVT stands for Open, High, Low, Close, Volume and Trades
        df['sma60000'] = df[1].rolling(window=60000).mean()
        df['sma30000'] = df[1].rolling(window=30000).mean()
        #calculate EMA on df
        df['ema12days'] = df[4].ewm(span=17280, adjust=False).mean()
        df['ema26days'] = df[4].ewm(span=37440, adjust=False).mean()
        df['macd'] = df['ema12days'] - df['ema26days']
        df.replace([np.inf, -np.inf], np.nan, inplace=True) 
        df.dropna(inplace=True) 
        self.df = df
        # print(self.df.head())
        # print(len(self.df))
        #len = 2675301
# env = TradingEnv()

In [14]:
# dfeth = pd.read_csv('./data/ETHUSD_1.csv', header=None)
# print(len(dfeth))
# dfeth.tail()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from keras.utils.generic_utils import get_custom_objects
import tensorflow as tf

from tensorflow.keras import backend as K
from tensorflow.keras.layers import Dense, Activation, Input
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.optimizers import Adam
import numpy as np

import gym, os
from gym import wrappers

from tensorflow.python.framework.ops import disable_eager_execution

disable_eager_execution()

physical_devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

tf.get_logger().setLevel('ERROR')
tf.config.experimental_run_functions_eagerly(False)
import pandas as pd
import numpy as np

import keras
import matplotlib.pyplot as plt
import seaborn as sns
from keras.utils.generic_utils import get_custom_objects

import keras
class Agent(object):
    def __init__(self, alpha, beta, gamma=0.90, n_actions=3,
                #  layer1_size=1024, layer2_size=512, input_dims=14):
                 layer1_size=1024, layer2_size=512, input_dims=14):
        self.gamma = gamma
        self.alpha = alpha
        self.beta = beta
        self.input_dims = input_dims
        self.fc1_dims = layer1_size
        self.fc2_dims = layer2_size
        self.n_actions = n_actions
        self.errorCount = 0
        # hier nieuw netwerk maken of oude inladen
        self.actor, self.critic, self.policy = self.build_actor_critic_network()
        # self.critic = keras.models.load_model('models/MACD/critic')
        # self.policy = keras.models.load_model('models/MACD/policy')
        # self.actor = load_model('models/MACD/actor')
        #unfreeze weights
        self.actor.trainable = True
        self.critic.trainable = True
        self.policy.trainable = True
        
        
        self.action_space = [i for i in range(n_actions)]
        self.probs = ""
    # def custom_loss(self,y_true, y_pred):
    #     out = K.clip(y_pred, 1e-8, 1-1e-8)
    #     log_lik = y_true*K.log(out)
    #     delta = Input(shape=[1])
    #     return K.sum(-log_lik* Input(shape=[1]))

    def build_actor_critic_network(self):
        NN_input = Input(shape=(self.input_dims,))
        delta = Input(shape=[1])
        dense1 = Dense(self.fc1_dims, activation='relu')(NN_input)
        dense2 = Dense(self.fc2_dims, activation='relu')(dense1)
        probs = Dense(self.n_actions, activation='softmax')(dense2)
        values = Dense(1, activation='linear')(dense2)

        

        actor = Model(inputs=[NN_input, delta], outputs=[probs])

        actor.compile(optimizer=Adam(lr=self.alpha), loss='categorical_crossentropy')

        critic = Model(inputs=[NN_input], outputs=[values])

        critic.compile(optimizer=Adam(lr=self.beta), loss='mean_squared_error')

        policy = Model(inputs=[NN_input], outputs=[probs])

        return actor, critic, policy

    def choose_action(self, observation):
        try:    
            state = observation[np.newaxis, :]
            probabilities = self.policy.predict(state)[0]
            self.probs = probabilities
            action = np.random.choice(self.action_space, p=probabilities)
            return action

        except Exception as ex:
            self.errorCount += 1
            print("counter", self.errorCount,"ex: ", ex)
            # print(probabilities)
            return 0


    def learn(self, state, action, reward, state_, done):
        state = state[np.newaxis,:]
        state_ = state_[np.newaxis,:]
        critic_value_ = self.critic.predict(state_)
        critic_value = self.critic.predict(state)

        target = reward + self.gamma*critic_value_*(1-int(done))
        delta =  target - critic_value

        actions = np.zeros([1, self.n_actions])
        actions[np.arange(1), action] = 1

        self.actor.fit([state, delta], actions, verbose=0)

        self.critic.fit(state, target, verbose=0) 
        
        
        
        
def train(agent2):
    agent = agent2
    env = TradingEnv()
    score_history = []
    
    steps = []
    actual_steps = []
    
    num_episodes = 1
    fullcounter = 0
    maxIterations = 9999999
    allEpisodesWorth = []
    allEpisodesPrice = []
    AllEpisodesSold = []
    AllEpisodesBought = []


    for i in range(num_episodes):
        agent.policy.save('models/MACD/policy')
        agent.critic.save('models/MACD/critic')
        agent.actor.save('models/MACD/actor')
        worth_history = []
        price_history = []
        bought = []
        sold = []
        done = False
        score = 0
        observation, done = env.reset()
        observation = np.asarray(observation)
        counter = 0
        lastBuy = -9000

        lastSold = -9999
        startprice = observation[0]
        maxIterations += 5000
        while not done:
            
            counter += 1
            fullcounter += 1
            # env.render()
            action = agent.choose_action(observation)
            actual_steps.append(action)
            if action == 1 or action == 2:
                # print("actual action ", action, "counter: ", counter,"probs ", agent.probs)
                pass
            
            if np.random.random() < .051:
                print("random action ", action, "counter: ", counter,"probs ", agent.probs)
                action = np.random.choice(3, p=[.7,.2,.1])
                # print("random action ", action, "counter: ", counter,"probs ", agent.probs)

            # if action == 1:
            #     if counter - lastBuy > 250 and counter - lastSold > 100:

            #         action = 1
            #         lastBuy = counter
            #     else:
            #         action = 0
            # if counter - lastBuy > 10000:
            #     action = 1
            # if action == 2 and counter - lastSold < 100:
            #     if counter - lastBuy > 200:
            #         action = 2
            #     else:
            #         action = 0
            # if counter > maxIterations:
            #     action = 2
            #     lastSold = counter
            #     sold.append(counter-1)
            # if env.invested > 15000 and action == 1: 
            #     action = 0
            # if env.invested < 100 and action == 2:
            #     action = 0
            # pick a number between 0 and 1
            # if the number is less than epsilon, pick a random action
            # if the number is greater than epsilon, pick the action with the highest q value


            observation_, reward, done = env.step(action, interval=5)
            steps.append(action)
            if action == 1 and reward > 0:
                lastBuy = counter
                bought.append(counter-1)
            elif action == 2 and reward != -10000 and reward != -2500:
                sold.append(counter-1)
            if action == 1 and reward > 0:
                # print("reward", reward,"action: ", action ," Episode: ", i, "Step: ", counter, "current_worth: ", env.calc_current_worth())
                pass
            if action == 2 and reward != -10000 and reward != -2500 :
                # print("reward", reward,"action: ", action ," Episode: ", i, "Step: ", counter, "current_worth: ", env.calc_current_worth())
                pass
            # if agent.probs[0] < .99:
            print("episode: ",i,"counter ", env.counter,"probas: ", agent.probs, " action ", action, "reward: ", reward,  "invested ", env.invested, "lastBuy: ", env.lastBuy, "worth", env.calc_current_worth() )
            observation_ = np.asarray(observation_)
            agent.learn(observation, action, reward, observation_, done)
            observation = observation_
            score += reward
            worth_history.append(env.calc_current_worth())
            price_history.append(env.current_price)
            if counter > maxIterations:
                done = True
        endprice = observation[0]
        
        allEpisodesWorth.append(worth_history)
        allEpisodesPrice.append(price_history)
        AllEpisodesSold.append(sold)
        AllEpisodesBought.append(bought)

        score_history.append(score)
        avg_score = np.mean(score_history[-100:])
        print('episode: ', i,'worth: %.2f' % env.calc_current_worth(),
              'avg score %.2f' % avg_score)
        print("1buy&sell: startworth:", 20000, "endworth:", (20000/startprice)*endprice)

    # save model progress
    agent.policy.save('models/MACD/policy')
    agent.critic.save('models/MACD/critic')
    agent.actor.save('models/MACD/actor')
    #make axes for 4 plots under eachother
    # print("allEpisodesWorth", len(allEpisodesWorth[0]))
    # print("allEpisodesPrice", len(allEpisodesPrice[0]))
    # print("AllEpisodesSold", len(AllEpisodesSold[0]))
    # print("AllEpisodesBought", len(AllEpisodesBought[0]))
    # print("startprice", startprice)
    # print("endprice", endprice)

    fig, axs = plt.subplots(len(allEpisodesWorth)+2, 1, figsize=(35, 50))
    #plot the price on first axis
    plot = sns.countplot(steps, ax=axs[0])
    plot.bar_label(plot.containers[0])
    plot2 = sns.countplot(actual_steps, ax=axs[1])
    plot2.bar_label(plot.containers[0])
    #set plotje on first axis
    # sns.lineplot(price_history,ax=axs[3], color="red", label="price")
    # sns.lineplot(worth_history,ax=axs[3], color="blue", label="investment_worth")


    # axs[1].plot(price_history)


    # axs[1].plot(price_history, color="red", label="price")
    # axs[1].plot(worth_history, color="blue", label="investment_worth")
    # #print shapes of bought and sold worth_history and price_history
    # if len(bought) > 0:
    #     axs[1].plot(bought, [worth_history[i] for i in bought], 'o', color='green')
    #     axs[1].plot(bought, [price_history[i] for i in bought], 'o', color='green', label="bought")
    # if len(sold) > 0:
    #     axs[1].plot(sold, [worth_history[i] for i in sold], 'o', color='pink')
    #     axs[1].plot(sold, [price_history[i] for i in sold], 'o', color='pink', label="sold")

    # axs[1].legend()
    # axs[1].set_title("Price and worth over time")
    # axs[1].set_xlabel("Time")
    # axs[1].set_ylabel("Price/Worth")
    #foreach episode plot the worth and price in one plot 
    for i in range(len(allEpisodesWorth)):
        axs[i+2].plot(allEpisodesWorth[i], color="blue", label="investment_worth")
        axs[i+2].plot(allEpisodesPrice[i], color="red", label="price")
        #print shapes of bought and sold worth_history and price_history
        try:
            if len(AllEpisodesBought[i]) > 0:
                axs[i+2].plot(AllEpisodesBought[i], [allEpisodesWorth[i][j] for j in AllEpisodesBought[i]], 'o', color='green')
                axs[i+2].plot(AllEpisodesBought[i], [allEpisodesPrice[i][j] for j in AllEpisodesBought[i]], 'o', color='green', label="bought")
            else:
                print("no bought")
            if len(AllEpisodesSold[i]) > 0:
                axs[i+2].plot(AllEpisodesSold[i], [allEpisodesWorth[i][j] for j in AllEpisodesSold[i]], 'o', color='purple')
                axs[i+2].plot(AllEpisodesSold[i], [allEpisodesPrice[i][j] for j in AllEpisodesSold[i]], 'o', color='purple', label="sold")
            else:
                print("no sold")
        except Exception as ex:
            print("error")
            print(ex)
            print(AllEpisodesBought[i])
            print(AllEpisodesSold[i])
            print(len(allEpisodesWorth[i]))
            print(len(allEpisodesPrice[i]))
            print(allEpisodesWorth[i])
            print(allEpisodesPrice[i])
            print("end error")
            break

        axs[i+2].legend()
        axs[i+2].set_title(f"ep: {i} Price and worth over time")
        axs[i+2].set_xlabel("Time 1interval = 5min")
        axs[i+2].set_ylabel("Price/Worth USD")

       

    fig.show()

    return agent
        

      
def demo(agent2):
    agent = agent2

    env = TradingEnv()
    score_history = []
    
    steps = []
    
    num_episodes = 1
    fullcounter = 0
    maxIterations = (262975 / 5) - 100

    allEpisodesWorth = []
    allEpisodesPrice = []
    AllEpisodesSold = []
    AllEpisodesBought = []


    for i in range(num_episodes):
        # agent.policy.save('models/MACD/policy')
        # agent.critic.save('models/MACD/critic')
        # agent.actor.save('models/MACD/actor')
        worth_history = []
        price_history = []
        unix_timestamps = []
        bought = []
        sold = []
        done = False
        score = 0
        observation, done = env.reset(demo=True)
        observation = np.asarray(observation)
        counter = 0
        lastBuy = -9000
        lastSold = -9999
        startprice = observation[0]
        while not done:
            unix_timestamps.append(env.current_unix_time)
            counter += 1
            fullcounter += 1
            action = agent.choose_action(observation)
            if np.random.random() < .005:
                action = np.random.choice(3, p=[.6,.3,.1])
            # if np.random.random() < .005:
            #     action = np.random.randint(0,3)
            
            
            steps.append(action)


            observation_, reward, done = env.step(action, interval=5)
            if action == 1 and reward >= 0:
                bought.append(counter-1)
            elif action == 2:
                sold.append(counter-1)
            if action == 2:
                # print("reward", reward,"action: ", action ," Episode: ", i, "Step: ", counter, "current_worth: ", env.calc_current_worth())
                pass
                
            observation_ = np.asarray(observation_)
            agent.learn(observation, action, reward, observation_, done)
            observation = observation_
            score += reward
            worth_history.append(env.calc_current_worth())
            price_history.append(env.current_price)
            if counter > maxIterations:
                done = True
            print("episode: ",i,"counter ", env.counter,"probas: ", agent.probs, " action ", action, "reward: ", reward,  "invested ", env.invested, "lastBuy: ", env.lastBuy, "worth", env.calc_current_worth() )
            
        endprice = observation[0]
        allEpisodesWorth.append(worth_history)
        allEpisodesPrice.append(price_history)
        AllEpisodesSold.append(sold)
        AllEpisodesBought.append(bought)

        score_history.append(score)
        avg_score = np.mean(score_history[-100:])
        print('episode: ', i,'worth: %.2f' % env.calc_current_worth(),
              'avg score %.2f' % avg_score)
        print("1buy&sell: startworth:", 20000, "endworth:", (20000/startprice)*endprice)

    # # save model progress
    # agent.policy.save('models/MACD/policy')
    # agent.critic.save('models/MACD/critic')
    # agent.actor.save('models/MACD/actor')
    #make axes for 4 plots under eachother
    # print("allEpisodesWorth", len(allEpisodesWorth[0]))
    # print("allEpisodesPrice", len(allEpisodesPrice[0]))
    # print("AllEpisodesSold", len(AllEpisodesSold[0]))
    # print("AllEpisodesBought", len(AllEpisodesBought[0]))
    # print("startprice", startprice)
    # print("endprice", endprice)

    fig, axs = plt.subplots(len(allEpisodesWorth)+1, 1, figsize=(35, 50))
    #plot the price on first axis
    plot = sns.countplot(steps, ax=axs[0])
    plot.bar_label(plot.containers[0])
    #set plotje on first axis
    # sns.lineplot(price_history,ax=axs[3], color="red", label="price")
    # sns.lineplot(worth_history,ax=axs[3], color="blue", label="investment_worth")


    # axs[1].plot(price_history)


    # axs[1].plot(price_history, color="red", label="price")
    # axs[1].plot(worth_history, color="blue", label="investment_worth")
    # #print shapes of bought and sold worth_history and price_history
    # if len(bought) > 0:
    #     axs[1].plot(bought, [worth_history[i] for i in bought], 'o', color='green')
    #     axs[1].plot(bought, [price_history[i] for i in bought], 'o', color='green', label="bought")
    # if len(sold) > 0:
    #     axs[1].plot(sold, [worth_history[i] for i in sold], 'o', color='pink')
    #     axs[1].plot(sold, [price_history[i] for i in sold], 'o', color='pink', label="sold")

    # axs[1].legend()
    # axs[1].set_title("Price and worth over time")
    # axs[1].set_xlabel("Time")
    # axs[1].set_ylabel("Price/Worth")
    #foreach episode plot the worth and price in one plot 
    for i in range(len(allEpisodesWorth)):
        axs[i+1].plot(allEpisodesWorth[i], color="blue", label="investment_worth")
        axs[i+1].plot(allEpisodesPrice[i], color="red", label="price")
        #print shapes of bought and sold worth_history and price_history
        try:
            if len(AllEpisodesBought[i]) > 0:
                axs[i+1].plot(AllEpisodesBought[i], [allEpisodesWorth[i][j] for j in AllEpisodesBought[i]], 'o', color='green')
                axs[i+1].plot(AllEpisodesBought[i], [allEpisodesPrice[i][j] for j in AllEpisodesBought[i]], 'o', color='green', label="bought")
            else:
                print("no bought")
            if len(AllEpisodesSold[i]) > 0:
                axs[i+1].plot(AllEpisodesSold[i], [allEpisodesWorth[i][j] for j in AllEpisodesSold[i]], 'o', color='purple')
                axs[i+1].plot(AllEpisodesSold[i], [allEpisodesPrice[i][j] for j in AllEpisodesSold[i]], 'o', color='purple', label="sold")
            else:
                print("no sold")
        except Exception as ex:
            print("error")
            print(ex)
            print(AllEpisodesBought[i])
            print(AllEpisodesSold[i])
            print(len(allEpisodesWorth[i]))
            print(len(allEpisodesPrice[i]))
            print(allEpisodesWorth[i])
            print(allEpisodesPrice[i])
            print("end error")
            break
        import datetime
        axs[i+1].legend()
        axs[i+1].set_xticklabels([datetime.datetime.fromtimestamp(tm) for tm in unix_timestamps],
        rotation=50)
        axs[i+1].set_title(f"ep: {i} Price and worth over time")
        axs[i+1].set_xlabel("Time 1interval = 5min")
        axs[i+1].set_ylabel("Price/Worth USD")

       

    fig.show()


        


# agent = Agent(alpha=0.001, beta=0.005)
agent = train(agent)
agent = train(agent)
agent = train(agent)
agent = train(agent)
agent = train(agent)
agent = train(agent)
agent = train(agent)
agent = train(agent)



# agent = train(agent)
# # save model progress
agent.policy.save_weights('models/weights/MACD3/policy')
agent.critic.save_weights('models/weights/MACD3/critic')
agent.actor.save_weights('models/weights/MACD3/actor')

# demo(agent)





